## 使用example

In [3]:
import tensorflow as tf
from sionna.channel.apply_ofdm_channel import ApplyOFDMChannel  

# 定义输入
x_real = tf.random.normal([10, 2, 2, 14, 64], dtype=tf.float32)
x_imag = tf.random.normal([10, 2, 2, 14, 64], dtype=tf.float32)
x = tf.complex(x_real, x_imag)

h_freq_real = tf.random.normal([10, 2, 2, 2, 2, 14, 64], dtype=tf.float32)
h_freq_imag = tf.random.normal([10, 2, 2, 2, 2, 14, 64], dtype=tf.float32)
h_freq = tf.complex(h_freq_real, h_freq_imag)

no = tf.constant(0.01, dtype=tf.float32)  # 示例噪声功率

# 创建 ApplyOFDMChannel 实例
apply_ofdm_channel = ApplyOFDMChannel()

# 调用实例
y = apply_ofdm_channel((x, h_freq, no))

print(y)


2024-06-22 19:24:49.950679: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 19:24:50.007565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 19:24:50.007592: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 19:24:50.009129: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-22 19:24:50.018090: I tensorflow/core/platform/cpu_feature_guar

tf.Tensor(
[[[[[-2.93457747e+00+6.36516714e+00j -2.61688066e+00-1.92103922e+00j
     -1.33327171e-02-6.27764463e-01j ...
      1.99499416e+00+3.08747143e-01j  3.42071152e+00-3.44051361e-01j
      2.46785903e+00-6.70659542e-01j]
    [-8.74877572e-02+3.35849452e+00j -1.27026510e+00+1.03728151e+00j
     -3.19785535e-01+3.77926731e+00j ...
     -2.48740172e+00-6.67156315e+00j -4.89531755e-01-6.88654184e-01j
      2.45589525e-01+9.48807478e-01j]
    [-2.72799754e+00-1.34375143e+00j -8.50740194e-01+1.50000993e-02j
     -3.71591115e+00+2.48915434e+00j ...
      2.92614245e+00+1.92470908e+00j -2.14149356e+00-1.87503576e-01j
      1.54800868e+00+4.30147976e-01j]
    ...
    [-1.12837768e+00-5.94051301e-01j  2.03508949e+00-1.15449643e+00j
      9.55111206e-01+1.06768084e+00j ...
      3.11591291e+00-3.44372344e+00j -4.75732565e-01-3.22407389e+00j
      2.93745875e+00-9.98482704e-01j]
    [ 2.70306444e+00-1.64724481e+00j -1.60253596e+00-2.13640046e+00j
      1.37678280e-01+4.49916422e-01j ...
   

2024-06-22 19:24:52.969506: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:31:00.0, compute capability: 8.9


## explain （tf version）
这段代码定义了一个名为 `ApplyOFDMChannel` 的 TensorFlow Keras 层，用于在频域中应用单抽头 OFDM 信道响应。这是一个自定义的 Keras 层类，可以在 Keras 模型中使用。让我们逐行解释相关代码。

### 导入必要的库
```python
import tensorflow as tf
from sionna.utils import expand_to_rank
from .awgn import AWGN
```
- `tensorflow`: 深度学习库，用于定义和训练神经网络。
- `expand_to_rank`: 假设这是一个实用函数，用于扩展张量的维度。
- `AWGN`: 假设这是一个添加加性高斯白噪声的自定义类。

### 定义 ApplyOFDMChannel 类
```python
class ApplyOFDMChannel(tf.keras.layers.Layer):
    r"""ApplyOFDMChannel(add_awgn=True, dtype=tf.complex64, **kwargs)
    
    ...
    """
```
- `ApplyOFDMChannel` 继承自 Keras 的 `Layer` 类。
- 使用了 Keras 层的构造函数来初始化该类。

#### `__init__` 方法
```python
def __init__(self, add_awgn=True, dtype=tf.complex64, **kwargs):
    super().__init__(trainable=False, dtype=dtype, **kwargs)
    self._add_awgn = add_awgn
```
- `__init__` 方法是类的构造函数。
- `add_awgn`：布尔值，是否添加加性高斯白噪声。
- `dtype`：处理和输出的复杂数据类型，默认为 `tf.complex64`。
- `super().__init__`：调用父类的构造函数，并设置 `trainable=False` 表示该层不可训练。

#### `build` 方法
```python
def build(self, input_shape):
    if self._add_awgn:
        self._awgn = AWGN(dtype=self.dtype)
```
- `build` 方法在第一次使用该层时被调用。
- 如果 `add_awgn` 为 `True`，则初始化 `AWGN` 类的实例。

#### `call` 方法
```python
def call(self, inputs):
    if self._add_awgn:
        x, h_freq, no = inputs
    else:
        x, h_freq = inputs

    # Apply the channel response
    x = expand_to_rank(x, h_freq.shape.rank, axis=1)
    y = tf.reduce_sum(tf.reduce_sum(h_freq * x, axis=4), axis=3)

    # Add AWGN if requested
    if self._add_awgn:
        y = self._awgn((y, no))

    return y
```
- `call` 方法是层的核心逻辑。
- `inputs` 是一个元组，可以是 `(x, h_freq, no)` 或 `(x, h_freq)`：
  - `x`：OFDM 输入信号。
  - `h_freq`：频域信道响应。
  - `no`：噪声功率。
- 根据 `add_awgn` 的值，解包输入元组。
- `expand_to_rank`：扩展 `x` 的维度以匹配 `h_freq` 的秩（rank）。
- `h_freq * x`：对输入信号应用频域信道响应。
- `tf.reduce_sum(tf.reduce_sum(h_freq * x, axis=4), axis=3)`：沿特定轴求和以获得输出信号 `y`。
- 如果 `add_awgn` 为 `True`，则调用 `AWGN` 实例添加噪声。
- 返回经过信道和（可选）噪声处理后的输出信号 `y`。

### 总结
- `ApplyOFDMChannel` 类用于在频域中应用单抽头信道响应，并可选地添加噪声。
- 该类继承自 `tf.keras.layers.Layer`，包含 `__init__`、`build` 和 `call` 方法。
- `__init__` 方法初始化层的配置。
- `build` 方法在第一次使用该层时被调用，初始化 AWGN 类的实例。
- `call` 方法实现层的核心逻辑，对输入信号应用信道响应并可选地添加噪声。

## explain （pytorch version）

将 `ApplyOFDMChannel` 层从 TensorFlow 改写成 PyTorch 版本如下：



### 定义 ApplyOFDMChannel 类
```python
class ApplyOFDMChannel(nn.Module):
    def __init__(self, add_awgn=True, dtype=torch.complex64):
        super(ApplyOFDMChannel, self).__init__()
        self.add_awgn = add_awgn
        self.dtype = dtype

        if self.add_awgn:
            self.awgn = AWGN(dtype=self.dtype)

    def forward(self, inputs):
        if self.add_awgn:
            x, h_freq, no = inputs
        else:
            x, h_freq = inputs

        # Apply the channel response
        x = expand_to_rank(x, h_freq.dim(), axis=1)
        y = torch.sum(torch.sum(h_freq * x, dim=4), dim=3)

        # Add AWGN if requested
        if self.add_awgn:
            y = self.awgn((y, no))

        return y
```

### 测试代码
```python
# 定义输入
x_real = torch.randn([10, 2, 2, 14, 64], dtype=torch.float32)
x_imag = torch.randn([10, 2, 2, 14, 64], dtype=torch.float32)
x = torch.complex(x_real, x_imag)

h_freq_real = torch.randn([10, 2, 2, 2, 2, 14, 64], dtype=torch.float32)
h_freq_imag = torch.randn([10, 2, 2, 2, 2, 14, 64], dtype=torch.float32)
h_freq = torch.complex(h_freq_real, h_freq_imag)

no = torch.tensor(0.01, dtype=torch.float32)  # 示例噪声功率

# 创建 ApplyOFDMChannel 实例
apply_ofdm_channel = ApplyOFDMChannel()

# 调用实例
y = apply_ofdm_channel((x, h_freq, no))

print(y)
```

### 代码解释
1. **导入库**：
   - `torch` 和 `torch.nn` 是 PyTorch 的核心库。
   - `torch.nn.functional` 是用于定义操作和激活函数的库。

2. **定义 `expand_to_rank` 函数**：
   - 这个函数将张量 `x` 扩展到指定的目标秩 `target_rank`。

3. **定义 `AWGN` 类**：
   - `AWGN` 类添加加性高斯白噪声。
   - `forward` 方法将噪声添加到输入信号 `y` 中。

4. **定义 `ApplyOFDMChannel` 类**：
   - `__init__` 方法初始化层的配置，包括是否添加噪声和数据类型。
   - `forward` 方法实现层的核心逻辑，对输入信号应用信道响应并可选地添加噪声。

5. **测试代码**：
   - 定义输入信号 `x` 和频域信道响应 `h_freq`。
   - 创建 `ApplyOFDMChannel` 类的实例并调用 `forward` 方法。

这样就完成了将 `ApplyOFDMChannel` 从 TensorFlow 改写成 PyTorch 的代码。

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from channel.torch_version.awgn import AWGN
from my_sionna.channel.torch_version.utils import expand_to_rank

def expand_to_rank(x, target_rank, axis=1):
    while x.dim() < target_rank:
        x = x.unsqueeze(axis)
    return x

class AWGN(nn.Module):
    def __init__(self, dtype=torch.complex64):
        super(AWGN, self).__init__()
        self.dtype = dtype

    def forward(self, inputs):
        y, no = inputs
        noise = torch.randn_like(y, dtype=torch.float32)
        noise = (noise + 1j * torch.randn_like(y, dtype=torch.float32)) * torch.sqrt(no / 2)
        return y + noise.to(self.dtype)

ModuleNotFoundError: No module named 'channel'

In [6]:
class ApplyOFDMChannel(nn.Module):
    def __init__(self, add_awgn=True, dtype=torch.complex64):
        super(ApplyOFDMChannel, self).__init__()
        self.add_awgn = add_awgn
        self.dtype = dtype
        if self.add_awgn:
            self.awgn = AWGN(dtype=self.dtype)

    def forward(self, inputs):
        if self.add_awgn:
            x, h_freq, no = inputs
        else:
            x, h_freq = inputs

        # Apply the channel response
        x = expand_to_rank(x, h_freq.dim(), axis=1)
        y = torch.sum(torch.sum(h_freq * x, dim=4), dim=3)

        # Add AWGN if requested
        if self.add_awgn:
            y = self.awgn((y, no))

        return y

In [7]:
x_real = torch.randn([10, 2, 2, 14, 64], dtype=torch.float32)
x_imag = torch.randn([10, 2, 2, 14, 64], dtype=torch.float32)
x = torch.complex(x_real, x_imag)

h_freq_real = torch.randn([10, 2, 2, 2, 2, 14, 64], dtype=torch.float32)
h_freq_imag = torch.randn([10, 2, 2, 2, 2, 14, 64], dtype=torch.float32)
h_freq = torch.complex(h_freq_real, h_freq_imag)

no = torch.tensor(0.01, dtype=torch.float32)  # 示例噪声功率

# 创建 ApplyOFDMChannel 实例
apply_ofdm_channel = ApplyOFDMChannel()

# 调用实例
y = apply_ofdm_channel((x, h_freq, no))

print(y)

tensor([[[[[ 1.2214e-01-3.3774e+00j,  8.8962e-01-4.8661e+00j,
             3.9031e+00-9.5894e-01j,  ...,
            -2.9277e+00+5.4814e+00j,  1.2358e+00-1.0893e-02j,
            -1.1860e-03-3.7509e+00j],
           [-9.8256e-01-2.3139e+00j,  1.7472e+00-2.1038e+00j,
            -1.0796e+00+3.0327e+00j,  ...,
            -4.6478e+00+7.7571e-01j,  1.3371e-01+2.6362e-01j,
             3.2039e-01+1.4950e+00j],
           [-4.0763e-01-5.2665e+00j, -3.9152e+00-3.1180e+00j,
             1.0605e+00+1.8356e+00j,  ...,
             1.9641e+00+1.1942e+00j, -3.3914e+00-1.9216e+00j,
             7.4533e+00+6.4597e+00j],
           ...,
           [-3.9485e+00+8.3363e-01j, -3.5766e+00-4.1601e+00j,
            -9.3543e-01+1.0327e+00j,  ...,
            -1.6137e+00-4.5942e+00j,  1.3774e+00+6.5422e+00j,
            -2.4307e+00-9.7486e-02j],
           [-1.9937e+00-3.8672e+00j, -3.4080e+00-5.3430e-02j,
             7.6691e+00-1.7852e+00j,  ...,
            -2.4805e+00+3.9469e+00j, -2.5393e-02-1.9263e+00